### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [1]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [1]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


logger = logging.getLogger()
device_cpu = torch.device("cpu")

In [2]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [10]:
df = pd.read_csv('./data/bsv_chunk256_ba_mld_1201-1500.csv')

In [11]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 860
Nombre de commentaires dans le train_set: 3441


In [6]:
labels = df.columns[1:3].to_list() 
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [9]:
df_texts = pd.read_csv('./data/bsv_chunk256_raw_1001-1200.csv')
all_texts = df_texts['report_text'].to_list()
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2801


### Création de LMDataBunch

In [10]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=4, #was 16, even 8 won't do
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [11]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cpu,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=False) #was true with gpu

Some weights of CamembertForMaskedLM were not initialized from the model checkpoint at camembert-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
lm_learner.fit(epochs=2, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(142, 2.301709981031821)

In [13]:
lm_learner.validate()

{'loss': 0.11850780993700027, 'perplexity': 1.1258156299591064}

In [14]:
lm_learner.save_model()

In [15]:
del lm_learner

### Création de databunch pour la classification

In [3]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [4]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [5]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cpu, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=False,#True when is cuda
                        logging_steps=50)

Some weights of the model checkpoint at model/model_out were not used when initializing CamembertForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForMultiLabelSequenceClassification were not initialized from the model checkpoint at model/model_out and are newly initialized: ['classifier.dense.w

In [6]:
cl_learner.fit(epochs=5,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(2155, 0.2934769226190593)

In [7]:
cl_learner.validate()

{'loss': 0.38437828076658426,
 'fbeta': 0.3950258493423462,
 'roc_auc': 0.9113796040025547}

In [8]:
cl_learner.save_model()

### Prédictions

In [7]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [5]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.9428858160972595), ('disease', 0.03713587298989296)]

In [6]:
predictor.predict("election américane")

[('bioagressor', 0.021512597799301147), ('disease', 0.01901276223361492)]

In [12]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique
predictor.predict("mouche de la carotte :ajouter trichloronate")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.595981776714325), ('disease', 0.011578208766877651)]

In [17]:
#cas disease: 0, bioagressor: 0 - oïdium : pas d'intervention 
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.7588301301002502), ('bioagressor', 0.01855621486902237)]

In [16]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.9032171964645386), ('bioagressor', 0.03460371121764183)]

In [7]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('disease', 0.9704827070236206), ('bioagressor', 0.11312197148799896)]

In [8]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('bioagressor', 0.021949905902147293), ('disease', 0.01820199377834797)]

In [16]:
#cas disease: 1, bioagressor: 1 - #mildiou (removed)
predictor.predict("Pomme de te r re du 17 mai 2013 BS omme Ce - N° 9 N° 9 En résumé : Risque  au 17 mai : entrée en phase de risque pour les variétés sensibles pour 5 secteurs. Pour autant, le seuil de nuisibilité n’est pas atteint. Utilisation du modèle Mileos® (www.mileos.fr) Le BSV pomme de terre de la région Centre mobilise le modèle Mileos® qui se base sur le cycle épidémique de Phytophtora infestans. • Quand démarre le risque  ? Le suivi du nombre des générations de  est un bon indicateur pour connaître le début de la période à risque de cette maladie. En fonction de la sensibilité variétale, le risque démarre : - à la sortie de")

[('disease', 0.027921902015805244), ('bioagressor', 0.016035165637731552)]

In [10]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('bioagressor', 0.021660201251506805), ('disease', 0.0180667731910944)]

In [17]:
#cas disease: 0, bioagressor: 0 + mildiou
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.8798115253448486), ('bioagressor', 0.029656751081347466)]

In [12]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.021866289898753166), ('disease', 0.017439812421798706)]

In [13]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('bioagressor', 0.021755171939730644), ('disease', 0.017790012061595917)]

In [15]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('bioagressor', 0.02116847224533558), ('disease', 0.018581580370664597)]

In [18]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('disease', 0.88606196641922), ('bioagressor', 0.034433670341968536)]

In [21]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('bioagressor', 0.021399231627583504), ('disease', 0.018853485584259033)]

In [22]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.9492413401603699), ('disease', 0.041776448488235474)]

### Analyse the fine-tuned model

In [3]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [4]:
#evaluation on validation set
df_val = pd.read_csv('./data/val_set.csv')

#predictor.get_learner()

In [5]:
df_val.Disease = df_val.Disease.astype(int)
df_val.tail()

,Unnamed: 0,report_text,Bioagressor,Disease,hazard_list,source_name
855,2112,MILDIOU DU TOURNESOL (Plasmopara halstedii) No...,0,0,NaN,Note_Commune-Mildiou-Tournesol-2012_cle0b4a54
856,1146,. . . . . . . . . . . . . . . . . . . . . . . ...,0,0,NaN,draaf.centre-val-de-loire.agriculture.gouv.fr_...
857,3528,"conditions automnales et hivernales, particuli...",0,0,NaN,www.paysdelaloire.chambagri.fr_fileadmin_docum...
858,4062,teigne des crucifères (Plutella xylostella) à ...,1,0,#teigne des crucifères #mineuse,BSV_-_Legumes_no10-1_cle0f73f2
859,4246,– N ° rédigé par Hervé FRANCOIS - Chambre d’a...,0,1,#septoriose,20130430_bsv_grandescultures_13_cle4d3d84


In [6]:
batch_predictions = predictor.predict_batch(df_val.report_text.to_list())

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [7]:
batch_predictions[:10]

[[('bioagressor', 0.024611881002783775), ('disease', 0.015559783205389977)],
 [('bioagressor', 0.022255484014749527), ('disease', 0.018081091344356537)],
 [('bioagressor', 0.9774226546287537), ('disease', 0.9137246608734131)],
 [('bioagressor', 0.9835402369499207), ('disease', 0.454463392496109)],
 [('bioagressor', 0.9053354263305664), ('disease', 0.021073412150144577)],
 [('bioagressor', 0.021258292719721794), ('disease', 0.018573641777038574)],
 [('bioagressor', 0.9359539747238159), ('disease', 0.031498100608587265)],
 [('bioagressor', 0.021741380915045738), ('disease', 0.01758815161883831)],
 [('bioagressor', 0.9797319769859314), ('disease', 0.8557525277137756)],
 [('bioagressor', 0.023648018017411232), ('disease', 0.01610063947737217)]]

In [8]:
#dict(batch_predictions[1]).values()
list_y_pred = [ dict(pred) for pred in batch_predictions]
#list_y_pred[-5:]

In [9]:
df_y_pred = pd.DataFrame(list_y_pred, columns =['bioagressor', 'disease']) 
df_y_pred = df_y_pred.rename(columns={"bioagressor": "Bioagressor", "disease": "Disease"})
df_y_pred.tail()

,Bioagressor,Disease
855,0.059409,0.948146
856,0.020627,0.019491
857,0.010868,0.130579
858,0.677683,0.009041
859,0.094809,0.966370


In [40]:
df_y_pred.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.410655,0.259423
std,0.435770,0.382325
min,0.010639,0.005906
25%,0.021565,0.017868
50%,0.067273,0.025178
75%,0.938286,0.589650
max,0.984609,0.987217


In [10]:
df_y_real = pd.DataFrame(df_val, columns=['Bioagressor', 'Disease'])
df_y_real.tail()

,Bioagressor,Disease
855,0,0
856,0,0
857,0,0
858,1,0
859,0,1


In [41]:
df_y_real.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.366279,0.201163
std,0.482067,0.401103
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [11]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [42]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None) #If None, the scores for each class are returned

array([0.76454294, 0.68609865])

In [48]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5, average='weighted')

0.736733795482217

In [20]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.87619048, 0.88439306])

In [49]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.8790983606557377

In [21]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.81656805, 0.77272727])

In [50]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.8010261333874197

In [22]:
accuracy_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5)

0.7790697674418605

In [14]:
accuracy_score(y_true=df_y_real.Bioagressor.values, y_pred=df_y_pred.Bioagressor.values > 0.5)


0.8558139534883721

In [47]:
accuracy_score(y_true=df_y_real.Disease.values, y_pred=df_y_pred.Disease.values > 0.5 )


0.8953488372093024

In [44]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average=None)

array([0.80863876, 0.72491272])

In [51]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average='weighted')

0.7789571944130496

In [59]:
df_results = pd.concat([df_val, df_y_pred > 0.5], axis=1, ignore_index=True)

In [60]:
df_results.to_csv('predictions.csv')